# Project 7 

In this project, I will implement PCA and K-means clustering techniques in order to determine patterns in airplane delays. 

#### Step 1: Load the python libraries needed for this project 

In [70]:
import pandas as pd 
from matplotlib import pyplot as plt
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn import metrics
import numpy as np
import sklearn
import psycopg2 as psy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from pandas import DataFrame
import seaborn as sns
from sklearn.cluster import KMeans

%matplotlib inline

#### Step 2: Examine data 

In [71]:
PATH = '/Users/cguy/Downloads'
df = pd.read_csv(PATH + '/airports.csv')
df2 = pd.read_csv(PATH + '/Airport_operations.csv')
df3 = pd.read_csv(PATH + '/airport_cancellations.csv')

In [72]:
print df.columns
print df2.columns
print df3.columns


Index([u'Key', u'LocID', u'AP_NAME', u'ALIAS', u'Facility Type', u'FAA REGION',
       u'COUNTY', u'CITY', u'STATE', u'AP Type', u'Latitude', u'Longitude',
       u'Boundary Data Available'],
      dtype='object')
Index([u'airport', u'year', u'departures for metric computation',
       u'arrivals for metric computation', u'percent on-time gate departures',
       u'percent on-time airport departures', u'percent on-time gate arrivals',
       u'average_gate_departure_delay', u'average_taxi_out_time',
       u'average taxi out delay', u'average airport departure delay',
       u'average airborne delay', u'average taxi in delay',
       u'average block delay', u'average gate arrival delay'],
      dtype='object')
Index([u'Airport', u'Year', u'Departure Cancellations',
       u'Arrival Cancellations', u'Departure Diversions',
       u'Arrival Diversions'],
      dtype='object')


In [75]:
df.head()

,key,locid,ap_name,alias,facility_type,faa_region,county,city,state,ap_type,latitude,longitude,boundary_data_available
0,3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
1,5088.0,X64,PATILLAS,NaN,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.019330,No
2,2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
3,2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
4,2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


In [76]:
df2.head()

,airport,year,departures_for_metric_computation,arrivals_for_metric_computation,percent_on-time_gate_departures,percent_on-time_airport_departures,percent_on-time_gate_arrivals,average_gate_departure_delay,average_taxi_out_time,average_taxi_out_delay,average_airport_departure_delay,average_airborne_delay,average_taxi_in_delay,average_block_delay,average_gate_arrival_delay
0,ABQ,2004,53971,53818,0.8030,0.7809,0.7921,10.38,9.89,2.43,12.10,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.8140,0.7922,0.8001,9.60,9.79,2.29,11.20,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.40,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [77]:
df3.head()

,airport,year,departure_cancellations,arrival_cancellations,departure_diversions,arrival_diversions
0,ABQ,2004.0,242.0,235.0,71.0,46.0
1,ABQ,2005.0,221.0,190.0,61.0,33.0
2,ABQ,2006.0,392.0,329.0,71.0,124.0
3,ABQ,2007.0,366.0,304.0,107.0,45.0
4,ABQ,2008.0,333.0,300.0,79.0,42.0


### Problem statement / aim for this project

Aims:

- Analyze the operations of major airports around the country
- Understand the characteristics and groupings of airports based on a dataset of departure and operational delays
- Understand the distribution, characteristics, and components of individual airports operations that are leading to these delays
- Modeling using principal component analysis
- Help FAA to cut down on delays nationwide
- Organize and store their data so that they can easily understand it after your consulting work is done

Problem Statment:

Which airports and operational characteristics should the FAA target to  decrease delays?

### Part 1: Create a PostgreSQL database 

#### 1. Create a database to house airport data

In [78]:
# Cleaning data up to save in postgres database. According to notes, postgres doesn't like capitals or spaces...

df.columns = [c.lower().replace(' ','_') for c in df.columns] 
df2.columns = [c.lower().replace(' ','_') for c in df2.columns]
df3.columns = [c.lower().replace(' ','_') for c in df3.columns]

print df.columns
print df2.columns
print df3.columns

Index([u'key', u'locid', u'ap_name', u'alias', u'facility_type', u'faa_region',
       u'county', u'city', u'state', u'ap_type', u'latitude', u'longitude',
       u'boundary_data_available'],
      dtype='object')
Index([u'airport', u'year', u'departures_for_metric_computation',
       u'arrivals_for_metric_computation', u'percent_on-time_gate_departures',
       u'percent_on-time_airport_departures', u'percent_on-time_gate_arrivals',
       u'average_gate_departure_delay', u'average_taxi_out_time',
       u'average_taxi_out_delay', u'average_airport_departure_delay',
       u'average_airborne_delay', u'average_taxi_in_delay',
       u'average_block_delay', u'average_gate_arrival_delay'],
      dtype='object')
Index([u'airport', u'year', u'departure_cancellations',
       u'arrival_cancellations', u'departure_diversions',
       u'arrival_diversions'],
      dtype='object')


In [9]:
engine = create_engine('postgresql://localhost:5432')

In [13]:
df.to_sql('airport_info', engine)
df2.to_sql('airport_ops', engine)
df3.to_sql('airport_cancels', engine)

In [10]:
# %load_ext sql
%reload_ext sql

/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [14]:
%%sql postgresql://localhost:5432/
        
SELECT * FROM airport_info LIMIT 5;

5 rows affected.


index,key,locid,ap_name,alias,facility_type,faa_region,county,city,state,ap_type,latitude,longitude,boundary_data_available
0,3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
1,5088.0,X64,PATILLAS,None,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.01933,No
2,2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
3,2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
4,2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


In [28]:
%%sql 

SELECT COUNT(*) FROM airport_info LIMIT 5;

1 rows affected.


count
5167


In [15]:
%%sql 

SELECT * FROM airport_ops LIMIT 5;

5 rows affected.


index,airport,year,departures_for_metric_computation,arrivals_for_metric_computation,percent_on-time_gate_departures,percent_on-time_airport_departures,percent_on-time_gate_arrivals,average_gate_departure_delay,average_taxi_out_time,average_taxi_out_delay,average_airport_departure_delay,average_airborne_delay,average_taxi_in_delay,average_block_delay,average_gate_arrival_delay
0,ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [27]:
%%sql 

SELECT COUNT(*) FROM airport_ops LIMIT 5;

1 rows affected.


count
841


In [16]:
%%sql 

SELECT * FROM airport_cancels LIMIT 5;

5 rows affected.


index,airport,year,departure_cancellations,arrival_cancellations,departure_diversions,arrival_diversions
0,ABQ,2004.0,242.0,235.0,71.0,46.0
1,ABQ,2005.0,221.0,190.0,61.0,33.0
2,ABQ,2006.0,392.0,329.0,71.0,124.0
3,ABQ,2007.0,366.0,304.0,107.0,45.0
4,ABQ,2008.0,333.0,300.0,79.0,42.0


In [17]:
%%sql 

SELECT COUNT(*) FROM airport_cancels LIMIT 5;

1 rows affected.


count
805


In [86]:
# Tables don't match up. Need to join these tables based on generated column (concatenation of airport and year)

In [36]:
%%sql 

ALTER TABLE airport_ops ADD COLUMN ap_yr_index VARCHAR;

Done.
(psycopg2.ProgrammingError) syntax error at or near ";"
LINE 1: UPDATE airport_ops;
                          ^
 [SQL: 'UPDATE airport_ops;']


In [40]:
%%sql

UPDATE airport_ops
SET ap_yr_index = airport || year;

841 rows affected.


[]

In [41]:
%%sql 

SELECT * FROM airport_ops LIMIT 5;

5 rows affected.


index,airport,year,departures_for_metric_computation,arrivals_for_metric_computation,percent_on-time_gate_departures,percent_on-time_airport_departures,percent_on-time_gate_arrivals,average_gate_departure_delay,average_taxi_out_time,average_taxi_out_delay,average_airport_departure_delay,average_airborne_delay,average_taxi_in_delay,average_block_delay,average_gate_arrival_delay,ap_yr_index
0,ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87,ABQ2004
1,ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24,ABQ2005
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82,ABQ2006
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71,ABQ2007
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48,ABQ2008


In [21]:
# Adding concatenated column to airport_cancels (for later merge with airport_ops...)

### Needed to convert year column from float to integer/no decimals. Couldn't figure out; Will do later....

In [79]:
# %%sql

# UPDATE airport_cancels 
# SET year = CAST(to_char(year, 9999.0));

In [80]:
# %%sql
# SELECT * FROM airport_cancels LIMIT 5;

### Query the database for intial data

In [24]:
# total # of cities

In [81]:
%%sql 

SELECT COUNT(DISTINCT city) FROM airport_info;

1 rows affected.


count
3745


In [26]:
%%sql 

SELECT COUNT(city) FROM airport_info;

1 rows affected.


count
5164


In [27]:
#table below also shows # of airports per city, I believe

In [57]:

%%sql 

SELECT city, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY city 
ORDER BY airports DESC
LIMIT 10;

10 rows affected.


city,airports
HOUSTON,18
ATLANTA,12
WASHINGTON,11
COLUMBUS,11
GREENVILLE,10
WASILLA,10
CLINTON,10
JACKSON,9
COLUMBIA,9
INDIANAPOLIS,9


In [29]:
# total # of states

In [30]:
%%sql 

SELECT COUNT(DISTINCT state) FROM airport_info;

1 rows affected.


count
54


In [31]:
%%sql 

SELECT COUNT(state) FROM airport_info;

1 rows affected.


count
5164


In [32]:
#table below also shows # of airports per state, I believe

In [58]:
%%sql 

SELECT state, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY state 
ORDER BY airports DESC
LIMIT 10;

10 rows affected.


state,airports
TX,393
AK,388
CA,254
MI,229
OH,169
MN,151
NY,140
OK,139
KS,138
WA,137


In [34]:
# total # of counties

In [35]:
%%sql 

SELECT COUNT(DISTINCT county) FROM airport_info;

1 rows affected.


count
1577


In [36]:
%%sql 

SELECT COUNT(county) FROM airport_info;

1 rows affected.


count
5164


In [37]:
#table below also shows # of airports per county, I believe

In [59]:
%%sql 

SELECT county, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY county 
ORDER BY airports DESC
LIMIT 10;

10 rows affected.


county,airports
WASHINGTON,51
LINCOLN,38
JEFFERSON,33
FRANKLIN,33
YUKON-KOYUKUK,29
JACKSON,29
MONTGOMERY,29
BRISTOL BAY,29
BETHEL,28
CLARK,28


In [39]:
# total # of airports

In [40]:
%%sql 

SELECT COUNT(DISTINCT ap_name) FROM airport_info;

1 rows affected.


count
4956


In [41]:
%%sql 

SELECT COUNT(ap_name) FROM airport_info;

1 rows affected.


count
5164


In [42]:
# of times airport shows up in dataset

In [60]:
%%sql 

SELECT ap_name, COUNT(*) AS airports 
FROM airport_info 
GROUP BY ap_name 
ORDER BY airports DESC
LIMIT 10;

10 rows affected.


ap_name,airports
JACKSON COUNTY,5
MONROE COUNTY,5
PLYMOUTH MUNI,4
ARLINGTON MUNI,4
BUFFALO MUNI,3
FRANKLIN COUNTY,3
CLEVELAND MUNI,3
CORNING MUNI,3
None,3
SHELBY COUNTY,3


In [44]:
# total # of locid

In [45]:
%%sql 

SELECT COUNT(DISTINCT locid) FROM airport_info;

1 rows affected.


count
5152


In [46]:
%%sql 

SELECT COUNT(locid) FROM airport_info;

1 rows affected.


count
5152


In [47]:
#table below shows that locid = airport (in other two tables)

In [65]:
%%sql 

SELECT locid, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY locid 
ORDER BY airports DESC
LIMIT 10;

10 rows affected.


locid,airports
None,12
00F,1
00W,1
00N,1
00R,1
00S,1
00M,1
01A,1
00C,1
01B,1


In [49]:
# total # of ap_types

In [50]:
%%sql 

SELECT COUNT(DISTINCT ap_type) FROM airport_info;

1 rows affected.


count
2


In [51]:
%%sql 

SELECT COUNT(ap_type) FROM airport_info;

1 rows affected.


count
5164


In [52]:
#table below also shows # of airports per ap_types, I believe

In [62]:
%%sql 

SELECT ap_type, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY ap_type 
ORDER BY airports DESC;

3 rows affected.


ap_type,airports
Public Use,4671
Federalized/Commercial,288
None,0


In [54]:
# total # of faa_region

In [55]:
%%sql 

SELECT COUNT(DISTINCT faa_region) FROM airport_info;

1 rows affected.


count
9


In [56]:
%%sql 

SELECT COUNT(faa_region) FROM airport_info;

1 rows affected.


count
5164


In [57]:
#table below also shows # of airports per faa_region, I believe

In [63]:
%%sql 

SELECT faa_region, COUNT(DISTINCT ap_name) AS airports 
FROM airport_info 
GROUP BY faa_region 
ORDER BY airports DESC;

10 rows affected.


faa_region,airports
AGL,1056
ASW,763
ASO,733
ANM,644
ACE,468
AEA,465
AWP,397
AAL,388
ANE,179
None,0


In [59]:
# total # of facility_type

In [60]:
%%sql 

SELECT COUNT(DISTINCT facility_type) FROM airport_info;

1 rows affected.


count
1


In [61]:
%%sql 

SELECT COUNT(facility_type) FROM airport_info;

1 rows affected.


count
5164


In [62]:
#table below proves I can drop this column

In [63]:
%%sql 

SELECT facility_type, COUNT(*) AS airports 
FROM airport_info 
GROUP BY facility_type 
ORDER BY airports DESC;

2 rows affected.


facility_type,airports
Airport,5164
None,3


In [64]:
# total # of key

In [65]:
%%sql 

SELECT COUNT(DISTINCT key) FROM airport_info;

1 rows affected.


count
5164


In [66]:
%%sql 

SELECT COUNT(key) FROM airport_info;

1 rows affected.


count
5164


In [67]:
#table below also shows # of airports per key, I believe

In [68]:
%%sql 

SELECT key, COUNT(*) AS airports 
FROM airport_info 
GROUP BY key 
ORDER BY airports DESC 
LIMIT 4;

4 rows affected.


key,airports
None,3
2291.0,1
4717.0,1
2186.0,1


In [69]:
# total # of boundary_data_available

In [70]:
%%sql 

SELECT COUNT(DISTINCT boundary_data_available) FROM airport_info;

1 rows affected.


count
2


In [71]:
%%sql 

SELECT COUNT(boundary_data_available) FROM airport_info;

1 rows affected.


count
5164


In [72]:
#table below also shows # of airports per boundary_data_available, I believe

In [73]:
%%sql 

SELECT boundary_data_available, COUNT(*) AS airports 
FROM airport_info 
GROUP BY boundary_data_available 
ORDER BY airports DESC;

3 rows affected.


boundary_data_available,airports
Yes,3498
No,1666
None,3


In [74]:
# total # of airports in airport ops dataset

In [75]:
%%sql 

SELECT COUNT(DISTINCT airport) FROM airport_ops;

1 rows affected.


count
77


In [76]:
%%sql 

SELECT COUNT(airport) FROM airport_ops;

1 rows affected.


count
841


In [77]:
#table below also shows # of years of data per airport, I believe (airport ops dataset)

In [84]:
%%sql 

SELECT airport, COUNT(*) AS No_years 
FROM airport_ops 
GROUP BY airport 
ORDER BY No_Years
LIMIT 10;

10 rows affected.


airport,no_years
SMF,8
PSP,8
SJC,11
ANC,11
IAD,11
BOS,11
HOU,11
EWR,11
PIT,11
BNA,11


In [79]:
# total # of years in airport ops dataset

In [80]:
%%sql 

SELECT COUNT(DISTINCT year) FROM airport_ops;

1 rows affected.


count
11


In [81]:
%%sql 

SELECT COUNT(year) FROM airport_ops;

1 rows affected.


count
841


In [82]:
#table below also shows # of airports per year in airport ops dataset

In [83]:
%%sql 

SELECT year, COUNT(*) AS number 
FROM airport_ops 
GROUP BY year 
ORDER BY year DESC;

11 rows affected.


year,number
2014,77
2013,77
2012,77
2011,77
2010,77
2009,77
2008,77
2007,77
2006,75
2005,75


In [84]:
# total # of airports in airport_cancels dataset

In [85]:
%%sql 

SELECT COUNT(DISTINCT airport) FROM airport_cancels;

1 rows affected.


count
74


In [86]:
%%sql 

SELECT COUNT(airport) FROM airport_cancels;

1 rows affected.


count
805


In [87]:
#table below also shows # of years per airport in airport cancels dataset

In [85]:
%%sql 

SELECT airport, COUNT(*) AS No_years 
FROM airport_cancels 
GROUP BY airport 
ORDER BY No_years
LIMIT 10;

10 rows affected.


airport,no_years
RFD,6
OXR,7
SJC,11
IAD,11
ANC,11
EWR,11
HOU,11
BOS,11
PIT,11
BNA,11


In [89]:
#table below also shows # of airports per year of data in airport cancels dataset

In [90]:
%%sql 

SELECT year, COUNT(*) AS number 
FROM airport_cancels 
GROUP BY year 
ORDER BY year DESC;

11 rows affected.


year,number
2014.0,72
2013.0,73
2012.0,73
2011.0,73
2010.0,73
2009.0,73
2008.0,74
2007.0,74
2006.0,74
2005.0,73


#### 1.2 Risks and assumptions of data

Risks: 
- Two tables (airport_ops and airport_cancels) don't match
- Do not have 11 years worth of data for all airports

Assumptions: 
- Data is normally distributed (will look into this during EDA) 
- Not a lot of missing values (so will be able to drop these NaNs)